In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from import_casa import casa
from casa import Cadet, Cadence
import re
import pandas as pd
import numpy as np
from transformers import BertForSequenceClassification, BertTokenizer
import torch

/Users/seantyh/anaconda3/lib/python3.7/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
cadet = Cadet.load(casa.get_data_path()/"cadet/op20")

[INFO] 2021-04-28 13:45:05,034 gensim.utils: loading KeyedVectors object from /Users/seantyh/langon/casa/data/cadet/op20/ft-2020.kv
[INFO] 2021-04-28 13:45:05,322 gensim.utils: setting ignored attribute vectors_norm to None
[INFO] 2021-04-28 13:45:05,323 gensim.utils: setting ignored attribute vectors_vocab_norm to None
[INFO] 2021-04-28 13:45:05,324 gensim.utils: setting ignored attribute vectors_ngrams_norm to None
[INFO] 2021-04-28 13:45:05,324 gensim.utils: setting ignored attribute buckets_word to None
[INFO] 2021-04-28 13:45:05,463 gensim.utils: FastTextKeyedVectors lifecycle event {'fname': '/Users/seantyh/langon/casa/data/cadet/op20/ft-2020.kv', 'datetime': '2021-04-28T13:45:05.442896', 'gensim': '4.0.0', 'python': '3.7.4 (default, Aug 13 2019, 15:17:50) \n[Clang 4.0.1 (tags/RELEASE_401/final)]', 'platform': 'Darwin-20.3.0-x86_64-i386-64bit', 'event': 'loaded'}
[WARNING] 2021-04-28 13:45:05,470 gensim.models.keyedvectors: destructive init_sims(replace=True) deprecated & no long

In [ ]:
ckpt_path = casa.get_data_path() / "cadence/bert-sentiment-50.02.1/model.ckpt"
model = BertForSequenceClassification.from_pretrained("bert-base-chinese", num_labels = 3)
model.load_state_dict(torch.load(ckpt_path, map_location="cpu"))
tokenizer = BertTokenizer.from_pretrained("bert-base-chinese")

In [6]:
import pandas as pd
cx_path = casa.get_data_path() / "cadence/sentiment-constructicon.csv"
cx_df = pd.read_csv(cx_path, index_col=None).dropna()
cx_list = cx_df.to_dict(orient='list')
cx_list = [(cx, score) for cx, score in zip(*cx_list.values())]

[INFO] 2021-04-28 13:47:29,718 numexpr.utils: Note: NumExpr detected 12 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 8.
[INFO] 2021-04-28 13:47:29,719 numexpr.utils: NumExpr defaulting to 8 threads.


## Cadence: **C**ht **A**bsa **DE**teciton with **N**eural network and **C**onstructicon **E**nsemble


## Samples

In [7]:
cadence = Cadence.build_Q2(cadet, model, tokenizer, cx_list)

In [8]:
cadence.analyze("中華在山上的訊號還不錯", summary=True)

'中華電信(0.94)/通訊品質(0.49)/Positive(0.91, Bert)'

In [9]:
cadence.analyze("只有遠傳沒有訊號", summary=True)

'遠傳電信(0.49)/通訊品質(0.96)/Negative(1.00, CxG)'

In [10]:
cadence.analyze("遠傳測速是在測心酸的", summary=False)

{'entity': ['中華電信', '台灣大哥大', '遠傳電信', '台灣之星', '亞太電信', '其他電信'],
 'entity_probs': array([0.01198956, 0.01227514, 0.94205485, 0.01115532, 0.01108392,
        0.01144121]),
 'service': ['資費方案', '通訊品質', '加值服務', '其他'],
 'service_probs': array([7.82737379e-05, 9.99760666e-01, 8.33699990e-05, 7.76904188e-05]),
 'seeds': ['測試', 'Speed', '測速', '上傳', '下載'],
 'seed_probs': array([0.95608653, 0.00644206, 0.00644206, 0.00644206, 0.00644206]),
 'sentiment': ['Neutral', 'Positive', 'Negative'],
 'sentiment_src': 'CxG',
 'sentiment_probs': array([0., 0., 1.], dtype=float32)}

In [11]:
cadence.analyze("中華撿到槍", summary=True)

'中華電信(0.94)/資費方案(0.44)/Positive(0.51, Bert)'